In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sqlalchemy
from sqlalchemy import create_engine # database connection
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, clear_output
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from datetime import datetime
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
# engine  = create_engine("sqlite:///../input/database.sqlite")
with sqlite3.connect('database.sqlite') as engine:
#    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 and season in ("2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016");'
#                                          , engine)
    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 ;', engine)


matches = matches[matches.columns[:11]]
teams = pd.read_sql_query('SELECT * FROM Team;', engine)

# Add team names & tidy up
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id', 'home_team_api_id', 'id_y', 'team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'away_team_api_id','team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'away_team'}, inplace=True)

matches.tail(20)
# matches.info()

,match_id,season,stage,date,home_team_goal,away_team_goal,team_fifa_api_id_x,home_team,team_fifa_api_id_y,away_team
3020,4749,2015/2016,8,2015-10-04 00:00:00,3,0,1.0,Arsenal,11.0,Manchester United
3021,4750,2015/2016,8,2015-10-03 00:00:00,0,1,2.0,Aston Villa,1806.0,Stoke City
3022,4751,2015/2016,8,2015-10-03 00:00:00,1,1,1943.0,Bournemouth,1795.0,Watford
3023,4752,2015/2016,8,2015-10-03 00:00:00,1,3,5.0,Chelsea,17.0,Southampton
3024,4753,2015/2016,8,2015-10-03 00:00:00,2,0,1799.0,Crystal Palace,109.0,West Bromwich Albion
3025,4754,2015/2016,8,2015-10-04 00:00:00,1,1,7.0,Everton,9.0,Liverpool
3026,4755,2015/2016,8,2015-10-03 00:00:00,6,1,10.0,Manchester City,13.0,Newcastle United
3027,4756,2015/2016,8,2015-10-03 00:00:00,1,2,1792.0,Norwich City,95.0,Leicester City
3028,4757,2015/2016,8,2015-10-03 00:00:00,2,2,106.0,Sunderland,19.0,West Ham United
3029,4758,2015/2016,8,2015-10-04 00:00:00,2,2,1960.0,Swansea City,18.0,Tottenham Hotspur


In [3]:
def convert_team_name(last_season):
    for index, row in last_season.iterrows():
        if row['home_team'] == 'Man United':
            last_season.loc[index, 'home_team'] = 'Manchester United'
        if row['home_team'] == 'Man City':
            last_season.loc[index, 'home_team'] = 'Manchester City'
        if row['home_team'] == 'Leicester':
            last_season.loc[index, 'home_team'] = 'Leicester City'
        if row['home_team'] == 'West Brom':
            last_season.loc[index, 'home_team'] = 'West Bromwich Albion'
        if row['home_team'] == 'Newcastle':
            last_season.loc[index, 'home_team'] = 'Newcastle United'
        if row['home_team'] == 'Stoke':
            last_season.loc[index, 'home_team'] = 'Stoke City'
        if row['home_team'] == 'Swansea':
            last_season.loc[index, 'home_team'] = 'Swansea City'
        if row['home_team'] == 'West Ham':
            last_season.loc[index, 'home_team'] = 'West Ham United'
        if row['home_team'] == 'Tottenham':
            last_season.loc[index, 'home_team'] = 'Tottenham Hotspur'
        if row['home_team'] == 'Hull':
            last_season.loc[index, 'home_team'] = 'Hull City'
        if row['home_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'home_team'] = 'Brighton'
        if row['home_team'] == 'Huddersfield Town':
            last_season.loc[index, 'home_team'] = 'Huddersfield'

        if row['away_team'] == 'Man United':
            last_season.loc[index, 'away_team'] = 'Manchester United'
        if row['away_team'] == 'Man City':
            last_season.loc[index, 'away_team'] = 'Manchester City'
        if row['away_team'] == 'Leicester':
            last_season.loc[index, 'away_team'] = 'Leicester City'
        if row['away_team'] == 'West Brom':
            last_season.loc[index, 'away_team'] = 'West Bromwich Albion'
        if row['away_team'] == 'Newcastle':
            last_season.loc[index, 'away_team'] = 'Newcastle United'
        if row['away_team'] == 'Stoke':
            last_season.loc[index, 'away_team'] = 'Stoke City'
        if row['away_team'] == 'Swansea':
            last_season.loc[index, 'away_team'] = 'Swansea City'
        if row['away_team'] == 'West Ham':
            last_season.loc[index, 'away_team'] = 'West Ham United'
        if row['away_team'] == 'Tottenham':
            last_season.loc[index, 'away_team'] = 'Tottenham Hotspur'
        if row['away_team'] == 'Hull':
            last_season.loc[index, 'away_team'] = 'Hull City'
        if row['away_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'away_team'] = 'Brighton'
        if row['away_team'] == 'Huddersfield Town':
            last_season.loc[index, 'away_team'] = 'Huddersfield'
    return last_season

def create_match_id():
    match_id_num = []
    match_id_num2 = []
    for i in range(1, 11):
        match_id_num.append(i)
    for i in match_id_num:
        match_id_num2.append(i + int(last_season[-1:]['match_id']))
    return match_id_num2

In [4]:
with sqlite3.connect('database2.sqlite') as con:
    matches_new = pd.read_sql('select * from match;', con)
    matches_schedule = pd.read_sql('select * from match_schedule;', con)
    team_rating = pd.read_sql('select * from team_rating;', con)

matches_new.tail(10)

,match_id,season,stage,date,home_team_goal,away_team_goal,home_team,away_team
694,695,2017/2018,32,07/04/18,2,2,Bournemouth,Crystal Palace
695,696,2017/2018,32,07/04/18,1,1,Brighton,Huddersfield
696,697,2017/2018,32,07/04/18,0,0,Everton,Liverpool
697,698,2017/2018,32,07/04/18,1,2,Leicester,Newcastle
698,699,2017/2018,32,07/04/18,2,3,Man City,Man United
699,700,2017/2018,32,07/04/18,1,2,Stoke,Tottenham
700,701,2017/2018,33,07/04/18,1,2,Watford,Burnley
701,702,2017/2018,33,07/04/18,1,1,West Brom,Swansea
702,703,2017/2018,33,08/04/18,3,2,Arsenal,Southampton
703,704,2017/2018,33,08/04/18,1,1,Chelsea,West Ham


In [8]:
# Parameters to change depending on season and wek we are running for
this_season = '2017/2018'
this_week = int(matches_data_17[-1:]['stage']) + 1
train_ratio = .8
model_weight = .8
this_week

30

In [9]:
matches_schedule = matches_schedule.loc[matches_schedule['stage'] == this_week]
matches_schedule['match_id'] = match_id
matches_schedule

,match_id,date,home_team,away_team,season,stage
290,7290,43190,Bournemouth,Tottenham Hotspur,2017/2018,30
291,7291,43190,Arsenal,Watford,2017/2018,30
292,7292,43190,Chelsea,Crystal Palace,2017/2018,30
293,7293,43190,Everton,Brighton & Hove Albion,2017/2018,30
294,7294,43190,Huddersfield Town,Swansea,2017/2018,30
295,7295,43190,Manchester United,Liverpool,2017/2018,30
296,7296,43190,Newcastle United,Southampton,2017/2018,30
297,7297,43190,Stoke City,Manchester City,2017/2018,30
298,7298,43190,West Bromwich Albion,Leicester City,2017/2018,30
299,7299,43190,West Ham United,Burnley,2017/2018,30


In [10]:
# Add to full training data to predict current season
matches = pd.concat([matches, last_season, matches_schedule])
matches = matches.reset_index(drop=True)
convert_team_name(matches)
# matches

# Create a full set of match data that can be used with feature engineering later
full_matches = matches.copy()
# full_matches.match_id.fillna(last_season['match_id'][-1:] + 1)
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
0,Newcastle United,1.0,2008-08-17 00:00:00,Manchester United,1.0,1729,2008/2009,1,11.0,13.0
1,West Bromwich Albion,0.0,2008-08-16 00:00:00,Arsenal,1.0,1730,2008/2009,1,1.0,109.0
2,Liverpool,1.0,2008-08-16 00:00:00,Sunderland,0.0,1731,2008/2009,1,106.0,9.0
3,Wigan Athletic,1.0,2008-08-16 00:00:00,West Ham United,2.0,1732,2008/2009,1,19.0,1917.0
4,Manchester City,2.0,2008-08-17 00:00:00,Aston Villa,4.0,1733,2008/2009,1,2.0,10.0
5,Blackburn Rovers,3.0,2008-08-16 00:00:00,Everton,2.0,1734,2008/2009,1,7.0,3.0
6,Tottenham Hotspur,1.0,2008-08-16 00:00:00,Middlesbrough,2.0,1735,2008/2009,1,12.0,18.0
7,Stoke City,1.0,2008-08-16 00:00:00,Bolton Wanderers,3.0,1736,2008/2009,1,4.0,1806.0
8,Fulham,1.0,2008-08-16 00:00:00,Hull City,2.0,1737,2008/2009,1,1952.0,144.0
9,Portsmouth,0.0,2008-08-17 00:00:00,Chelsea,4.0,1738,2008/2009,1,5.0,1790.0


In [11]:
full_matches.drop(matches[matches.season == '2008/2009'].index, inplace=True)
full_matches.drop(matches[matches.season == '2009/2010'].index, inplace=True)
full_matches.drop(matches[matches.season == '2010/2011'].index, inplace=True)
full_matches.drop(matches[matches.season == '2011/2012'].index, inplace=True)

full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0
1521,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0
1522,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0
1523,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0
1524,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0
1529,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0


full_matches : ข้อมูลการแข่งขันทั้งหมดรวมอาทิตย์ต่อไปด้วย

In [12]:
# State the result as home or away win/draw/lose - 6 possibilites
# Create a binary result
def determine_result(match_list):
    match_list['home_win'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)
    match_list['home_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['home_lose'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_win'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['away_lose'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)

# Set up the matches data how I need it

# Add binary feature for W/D/L home and away
determine_result(full_matches)

# Sort in date order
full_matches.sort_values(by='date', inplace=True)

full_matches.head()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1


new_team : ข้อมูลทีมที่เลื่อนชั้น พร้อมฤดูกาล

In [13]:
# Cope with newly promoted teams with limited or no stats
team_data = {'team':['West Bromwich Albion', 'Stoke City', 'Hull City',
                     'Wolverhampton Wanderers', 'Birmingham City', 'Burnley',
                     'Newcastle United', 'West Bromwich Albion', 'Blackpool',
                     'Queens Park Rangers', 'Norwich City', 'Swansea City',
                     'Reading', 'Southampton', 'West Ham United',
                     'Cardiff City', 'Crystal Palace', 'Hull City',
                     'Leicester City', 'Burnley', 'Queens Park Rangers',
                     'Bournemouth', 'Watford', 'Norwich City',
                     'Burnley', 'Middlesbrough', 'Hull City',
                     'Brighton', 'Newcastle United', 'Huddersfield'
                    ],
             'season':["2008/2009", "2008/2009", "2008/2009",
                       "2009/2010", "2009/2010", "2009/2010",
                       "2010/2011", "2010/2011", "2010/2011",
                       "2011/2012", "2011/2012", "2011/2012",
                       "2012/2013", "2012/2013", "2012/2013",
                       "2013/2014", "2013/2014", "2013/2014",
                       "2014/2015", "2014/2015", "2014/2015",
                       "2015/2016", "2015/2016", "2015/2016",
                       "2016/2017", "2016/2017", "2016/2017",
                       "2017/2018", "2017/2018", "2017/2018"
                      ]
            }
new_teams = pd.DataFrame(team_data, columns=['team', 'season'])
new_teams.head()

,team,season
0,West Bromwich Albion,2008/2009
1,Stoke City,2008/2009
2,Hull City,2008/2009
3,Wolverhampton Wanderers,2009/2010
4,Birmingham City,2009/2010


full_match_features : dummy ข้อมูลของสถิติเก่าๆ

In [14]:
full_match_features = pd.DataFrame(full_matches[['season', 'stage']])
                                   #,
                                   #columns=[['season', 'stage']])
full_match_features.head()

,season,stage
1520,2012/2013,1
1527,2012/2013,1
1526,2012/2013,1
1525,2012/2013,1
1528,2012/2013,1


In [15]:
team_rating

,team_rating_id,season,team_name,attack_rating,midfield_rating,defence_rating,team_rating
0,0,2012/2013,Manchester United,86,81,82,82
1,1,2012/2013,Manchester City,85,83,80,82
2,2,2012/2013,Chelsea,82,80,81,81
3,3,2012/2013,Tottenham Hotspur,82,78,79,80
4,4,2012/2013,Arsenal,80,79,80,80
5,5,2012/2013,Liverpool,76,80,79,79
6,6,2012/2013,Newcastle United,81,77,77,77
7,7,2012/2013,Fulham,78,75,75,77
8,8,2012/2013,Everton,74,76,77,76
9,9,2012/2013,Queens Park Rangers,77,73,74,75


In [16]:
team_rating.drop('team_rating_id', axis=1, inplace=True)
team_rating2 = team_rating.copy()
team_rating = team_rating.rename(columns = {'team_name': 'home_team'})
team_rating2 = team_rating2.rename(columns = {'team_name': 'away_team'})

In [17]:
full_matches = pd.merge(full_matches, team_rating, how='left', on=['season', 'home_team'])
full_matches = full_matches.rename(columns = {'attack_rating': 'home_team_attack_rating'})
full_matches = full_matches.rename(columns = {'midfield_rating': 'home_team_midfield_rating'})
full_matches = full_matches.rename(columns = {'defence_rating': 'home_team_defence_rating'})
full_matches = full_matches.rename(columns = {'team_rating': 'home_team_rating'})
full_matches = pd.merge(full_matches, team_rating2, how='left', on=['season', 'away_team'])
full_matches = full_matches.rename(columns = {'attack_rating': 'away_team_attack_rating'})
full_matches = full_matches.rename(columns = {'midfield_rating': 'away_team_midfield_rating'})
full_matches = full_matches.rename(columns = {'defence_rating': 'away_team_defence_rating'})
full_matches = full_matches.rename(columns = {'team_rating': 'away_team_rating'})
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_draw,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
0,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,...,1,0,80,79,80,80,75,73,74,75
1,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,...,0,1,74,74,71,74,76,80,79,79
2,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,...,1,0,72,69,71,71,74,74,75,75
3,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,...,0,0,77,73,74,75,74,73,73,74
4,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,...,0,1,75,74,72,74,74,72,71,75
5,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0,...,0,1,78,75,75,77,72,73,73,73
6,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0,...,0,1,81,77,77,77,82,78,79,80
7,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0,...,0,1,85,83,80,82,73,74,71,73
8,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0,...,0,0,74,72,72,74,82,80,81,81
9,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0,...,0,1,74,76,77,76,86,81,82,82


In [18]:
# Convert home & team into a binary feature, ie Arsenal_h or Arsenal_a
# Need all seasons data for team binary feature
full_match_features = pd.DataFrame(full_matches[['season', 'stage']]) # เอา column ไหนมาบ้าง
#                                    columns=[['season', 'stage']])

full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['home_team']).rename(columns=lambda x: str(x) + '_h')],
                                axis=1)
full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['away_team']).rename(columns=lambda x: str(x) + '_a')],
                         axis=1)

full_match_features.head() # ข้อมูลสถิติเก่าๆ

,season,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
0,2012/2013,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


full_match_features_2 : ข้อมูล rating team เก่าๆ

In [19]:
# DataFrame team rating
full_match_features_2 = pd.DataFrame(full_matches[['season', 'stage', 'home_team_attack_rating',
       'home_team_midfield_rating', 'home_team_defence_rating',
       'home_team_rating', 'away_team_attack_rating',
       'away_team_midfield_rating', 'away_team_defence_rating',
       'away_team_rating']])

full_match_features_2.head() # ข้อมูล team rating

,season,stage,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
0,2012/2013,1,80,79,80,80,75,73,74,75
1,2012/2013,1,74,74,71,74,76,80,79,79
2,2012/2013,1,72,69,71,71,74,74,75,75
3,2012/2013,1,77,73,74,75,74,73,73,74
4,2012/2013,1,75,74,72,74,74,72,71,75


In [20]:
# Function to determine whether the highest prediction is for win/draw/lose

def predict_home_result(match):
    if (match['win'] >= match['draw']) & (match['win'] >= match['lose']):
        return 'win' # Favour a home win if probability equal
    elif (match['lose'] > match['win']) & (match['lose'] > match['draw']):
        return 'lose'
    else:
        return 'draw'

train_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ train model 

In [21]:
# To predict this season (1, this week only, remove this week from training set
train_match_features = full_match_features.loc[:int(full_match_features.shape[0] * train_ratio )].copy()

train_match_features.drop(['season'], axis=1, inplace=True)
train_match_features.tail()

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1756,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1757,24,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1758,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1759,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1760,25,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


train_match_features_2 : ข้อมูล rating team เก่าๆ ไว้ train model

In [22]:
# To predict this season (1, this week only, remove this week from training set
train_match_features_2 = full_match_features_2.loc[:int(full_match_features.shape[0] * train_ratio  )].copy()

train_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
train_match_features_2.tail()

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
1756,78,78,76,77,74,74,76,74
1757,81,76,74,75,77,78,77,78
1758,83,85,81,83,78,76,75,77
1759,79,78,76,78,84,84,82,83
1760,81,83,82,83,76,74,75,75


In [23]:
# Get the target results for training

# Function to determine whether result is a win/draw/lose
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'

train_matches : ข้อมูล train ทั้งหมดพร้อม label แล้ว

In [24]:
#  Add the home team result column to the matches dataframe
full_matches['home_team_result'] = full_matches.apply(determine_home_result, axis=1)

# To predict this season, this week, remove latest week from training results
train_matches = full_matches.loc[:int(full_matches.shape[0] *  train_ratio  )].copy()

targets = train_matches['home_team_result'].values
train_matches.tail()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
1756,Burnley,1.0,42770,Watford,2.0,6236,2016/2017,24,NaN,NaN,...,1,78,78,76,77,74,74,76,74,win
1757,Stoke City,0.0,42770,West Bromwich Albion,1.0,6237,2016/2017,24,NaN,NaN,...,1,81,76,74,75,77,78,77,78,win
1758,Swansea City,1.0,42771,Manchester City,2.0,6239,2016/2017,24,NaN,NaN,...,1,83,85,81,83,78,76,75,77,win
1759,Manchester United,3.0,42771,Leicester City,0.0,6238,2016/2017,24,NaN,NaN,...,0,79,78,76,78,84,84,82,83,lose
1760,Hull City,0.0,42777,Arsenal,2.0,6240,2016/2017,25,NaN,NaN,...,1,81,83,82,83,76,74,75,75,win


test_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ test model

In [25]:
# Get the test matches in correct format:

# Predict this season this week
test_match_features = full_match_features.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features.drop(full_match_features.index[-10:], inplace=True)
test_match_features.drop(['season'], axis=1, inplace=True)
test_match_features

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1760,25,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1761,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1762,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1763,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1764,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1765,25,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1766,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1767,25,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1768,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1769,25,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


test_match_features_1 : ข้อมูล dummy ของ week ถัดไป ไว้ทำนายจาก model สถิติเก่า

In [26]:
test_match_features_1 = full_match_features.iloc[-10:].copy()
test_match_features_1.drop(['season'], axis=1, inplace=True)
test_match_features_1

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
2190,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2191,30,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2192,30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2193,30,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2194,30,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2195,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2196,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2197,30,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2198,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2199,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


test_match_features_2 : ข้อมูล team rating เก่าๆไว้ test model

In [27]:
test_match_features_2 = full_match_features_2.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features_2.drop(full_match_features.index[-10:], inplace=True)
test_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
1760,81,83,82,83,76,74,75,75
1761,82,81,81,81,85,81,81,81
1762,84,84,82,83,78,78,76,77
1763,76,74,74,75,85,78,79,80
1764,77,78,77,78,80,77,76,77
1765,81,76,73,76,78,76,77,78
1766,79,77,77,78,81,76,74,75
1767,74,74,76,74,86,84,81,83
1768,78,76,75,77,79,78,76,78
1769,74,75,74,74,83,85,81,83


test_match_features_2_1 : ข้อมูล next week ของ team rating ไว้ทำนายจาก model team rating

In [28]:
test_match_features_2_1 = full_match_features_2.iloc[-10:].copy()
test_match_features_2_1.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2_1

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
2190,78,77,77,77,79,79,76,78
2191,77,74,75,75,85,83,82,83
2192,84,82,81,82,79,77,75,77
2193,85,84,83,84,81,76,75,76
2194,80,80,77,80,74,75,74,75
2195,75,73,73,73,76,75,74,75
2196,86,84,81,84,84,81,79,81
2197,76,75,75,75,78,77,77,77
2198,77,77,76,77,85,87,82,84
2199,79,79,77,78,74,76,76,76


model_test_matches : ข้อมูลทั้งหมดไว้ test model

In [29]:
# Don't have the target results yet but I have entered dummy data of 0-0 draws. 
# Then re-run with actual scores after they are played for comparison
model_test_matches = full_matches.loc[int(full_matches.shape[0] * train_ratio  ):].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches = model_test_matches.reset_index(drop=True)
model_test_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
0,Hull City,0.0,42777,Arsenal,2.0,6240,2016/2017,25,NaN,NaN,...,1,81,83,82,83,76,74,75,75,win
1,Tottenham Hotspur,0.0,42777,Liverpool,2.0,6241,2016/2017,25,NaN,NaN,...,1,82,81,81,81,85,81,81,81,win
2,Watford,0.0,42777,Manchester United,2.0,6242,2016/2017,25,NaN,NaN,...,1,84,84,82,83,78,78,76,77,win
3,Everton,0.0,42777,Middlesbrough,0.0,6243,2016/2017,25,NaN,NaN,...,0,76,74,74,75,85,78,79,80,draw
4,Crystal Palace,0.0,42777,Stoke City,1.0,6244,2016/2017,25,NaN,NaN,...,1,77,78,77,78,80,77,76,77,win
5,Southampton,4.0,42777,Sunderland,0.0,6245,2016/2017,25,NaN,NaN,...,0,81,76,73,76,78,76,77,78,lose
6,West Bromwich Albion,2.0,42777,West Ham United,2.0,6246,2016/2017,25,NaN,NaN,...,0,79,77,77,78,81,76,74,75,draw
7,Chelsea,1.0,42778,Burnley,1.0,6247,2016/2017,25,NaN,NaN,...,0,74,74,76,74,86,84,81,83,draw
8,Leicester City,0.0,42778,Swansea City,2.0,6248,2016/2017,25,NaN,NaN,...,1,78,76,75,77,79,78,76,78,win
9,Manchester City,2.0,42779,Bournemouth,0.0,6249,2016/2017,25,NaN,NaN,...,0,74,75,74,74,83,85,81,83,lose


model_test_matches_1 : ข้อมูลทั้งหมดของ next week

In [30]:
model_test_matches_1 = full_matches.iloc[-10:].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

# model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches_1 = model_test_matches_1.reset_index(drop=True)
model_test_matches_1

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
0,Leicester City,NaN,43190,West Bromwich Albion,NaN,7298,2017/2018,30,NaN,NaN,...,0,78,77,77,77,79,79,76,78,draw
1,Tottenham Hotspur,NaN,43190,Bournemouth,NaN,7290,2017/2018,30,NaN,NaN,...,0,77,74,75,75,85,83,82,83,draw
2,Watford,NaN,43190,Arsenal,NaN,7291,2017/2018,30,NaN,NaN,...,0,84,82,81,82,79,77,75,77,draw
3,Crystal Palace,NaN,43190,Chelsea,NaN,7292,2017/2018,30,NaN,NaN,...,0,85,84,83,84,81,76,75,76,draw
4,Brighton,NaN,43190,Everton,NaN,7293,2017/2018,30,NaN,NaN,...,0,80,80,77,80,74,75,74,75,draw
5,Swansea City,NaN,43190,Huddersfield,NaN,7294,2017/2018,30,NaN,NaN,...,0,75,73,73,73,76,75,74,75,draw
6,Liverpool,NaN,43190,Manchester United,NaN,7295,2017/2018,30,NaN,NaN,...,0,86,84,81,84,84,81,79,81,draw
7,Southampton,NaN,43190,Newcastle United,NaN,7296,2017/2018,30,NaN,NaN,...,0,76,75,75,75,78,77,77,77,draw
8,Manchester City,NaN,43190,Stoke City,NaN,7297,2017/2018,30,NaN,NaN,...,0,77,77,76,77,85,87,82,84,draw
9,Burnley,NaN,43190,West Ham United,NaN,7299,2017/2018,30,NaN,NaN,...,0,79,79,77,78,74,76,76,76,draw


## Fit model

fit ของ ข้อมูลการแข่งขันเก่าๆ

In [31]:
# Train, then predict 
model = MultinomialNB()

model.fit(train_match_features.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted = model.predict_proba(test_match_features.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table = pd.DataFrame(predicted,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table['predict_res'] = predicted_table.apply(predict_home_result, axis=1)
predicted_table['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table[predicted_table['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.586047
lose           0.586047
win            0.586047
predict_res    0.586047
actual_res     0.586047
dtype: float64

test model ข้อมูลการแข่งขันเก่าๆ

In [32]:
predicted_1 = model.predict_proba(test_match_features_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_1 = pd.DataFrame(predicted_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_1['predict_res'] = predicted_table_1.apply(predict_home_result, axis=1)
predicted_table_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_1[predicted_table_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

fit ข้อมูล team rating เก่าๆ

In [33]:
# Train, then predict 
model_2 = MultinomialNB()

model_2.fit(train_match_features_2.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted_2 = model_2.predict_proba(test_match_features_2.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2 = pd.DataFrame(predicted_2,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2['predict_res'] = predicted_table_2.apply(predict_home_result, axis=1)
predicted_table_2['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2[predicted_table_2['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.576744
lose           0.576744
win            0.576744
predict_res    0.576744
actual_res     0.576744
dtype: float64

train model ข้อมูล team rating

In [34]:
predicted_2_1 = model_2.predict_proba(test_match_features_2_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2_1 = pd.DataFrame(predicted_2_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2_1['predict_res'] = predicted_table_2_1.apply(predict_home_result, axis=1)
predicted_table_2_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2_1[predicted_table_2_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

## Test data result from old match

In [35]:
compare_results = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results = pd.concat([compare_results, predicted_table], axis=1)
compare_results = compare_results[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results = compare_results.rename(columns={'predict_res': 'predict_result'})
compare_results.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
420,7288,29,1.0,0.0,Manchester City,win,Chelsea,0.161671,0.286001,0.552328,win
421,7287,29,2.0,1.0,Brighton,lose,Arsenal,0.275442,0.531092,0.193467,win
422,7289,29,2.0,3.0,Crystal Palace,lose,Manchester United,0.128672,0.705033,0.166295,lose
423,7286,29,1.0,0.0,Watford,win,West Bromwich Albion,0.341205,0.260395,0.398400,win
424,7285,29,2.0,0.0,Tottenham Hotspur,win,Huddersfield,0.276982,0.229879,0.493139,win
425,7282,29,2.0,0.0,Liverpool,win,Newcastle United,0.171722,0.092786,0.735492,win
426,7283,29,0.0,0.0,Southampton,win,Stoke City,0.286392,0.196502,0.517107,draw
427,7281,29,1.0,1.0,Leicester City,win,Bournemouth,0.203795,0.209409,0.586797,draw
428,7280,29,2.0,1.0,Burnley,win,Everton,0.324409,0.336551,0.339040,win
429,7284,29,4.0,1.0,Swansea City,win,West Ham United,0.296126,0.275907,0.427968,win


## Test data result from old team rating

In [36]:
compare_results_2 = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2 = pd.concat([compare_results_2, predicted_table_2], axis=1)
compare_results_2 = compare_results_2[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2 = compare_results_2.rename(columns={'predict_res': 'predict_result'})
compare_results_2.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
420,7288,29,1.0,0.0,Manchester City,win,Chelsea,0.259910,0.280327,0.459763,win
421,7287,29,2.0,1.0,Brighton,lose,Arsenal,0.272545,0.443202,0.284254,win
422,7289,29,2.0,3.0,Crystal Palace,lose,Manchester United,0.270995,0.425450,0.303555,lose
423,7286,29,1.0,0.0,Watford,win,West Bromwich Albion,0.261240,0.296544,0.442217,win
424,7285,29,2.0,0.0,Tottenham Hotspur,win,Huddersfield,0.204933,0.141605,0.653462,win
425,7282,29,2.0,0.0,Liverpool,win,Newcastle United,0.229550,0.191725,0.578726,win
426,7283,29,0.0,0.0,Southampton,win,Stoke City,0.259046,0.282536,0.458418,draw
427,7281,29,1.0,1.0,Leicester City,win,Bournemouth,0.248817,0.242127,0.509056,draw
428,7280,29,2.0,1.0,Burnley,win,Everton,0.271784,0.362904,0.365312,win
429,7284,29,4.0,1.0,Swansea City,win,West Ham United,0.269562,0.354320,0.376118,win


## Test data result from weight models

In [37]:
compare_results_3 = compare_results.copy()

# weight data
compare_results_3['draw'] = compare_results['draw'] * model_weight + compare_results_2['draw'] * (1 - model_weight)
compare_results_3['lose'] = compare_results['lose'] * model_weight + compare_results_2['lose'] * (1 - model_weight)
compare_results_3['win'] = compare_results['win'] * model_weight + compare_results_2['win'] * (1 - model_weight)
compare_results_3['predict_result'] = compare_results_3.apply(predict_home_result, axis=1)
compare_results_3.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
420,7288,29,1.0,0.0,Manchester City,win,Chelsea,0.181319,0.284867,0.533815,win
421,7287,29,2.0,1.0,Brighton,lose,Arsenal,0.274862,0.513514,0.211624,win
422,7289,29,2.0,3.0,Crystal Palace,lose,Manchester United,0.157137,0.649116,0.193747,lose
423,7286,29,1.0,0.0,Watford,win,West Bromwich Albion,0.325212,0.267625,0.407163,win
424,7285,29,2.0,0.0,Tottenham Hotspur,win,Huddersfield,0.262572,0.212224,0.525204,win
425,7282,29,2.0,0.0,Liverpool,win,Newcastle United,0.183288,0.112574,0.704139,win
426,7283,29,0.0,0.0,Southampton,win,Stoke City,0.280922,0.213708,0.505369,draw
427,7281,29,1.0,1.0,Leicester City,win,Bournemouth,0.212799,0.215952,0.571249,draw
428,7280,29,2.0,1.0,Burnley,win,Everton,0.313884,0.341822,0.344294,win
429,7284,29,4.0,1.0,Swansea City,win,West Ham United,0.290813,0.291589,0.417598,win


### Accuracy of train data

In [38]:
print(f"Accuracy of old match model : {model.score(train_match_features.values, targets) * 100:.2f}%")

print(f"Accuracy of team rating model : {model_2.score(train_match_features_2.values, targets) *100:.2f}%")

Accuracy of old match model : 54.86%
Accuracy of team rating model : 53.38%


## Accuracy score of each model

In [39]:
# evaluate accuracy of prediction
print('Accuracy of prediction (old match) = {:.2f}%'.format(accuracy_score(compare_results['actual_res'], compare_results['predict_result']) * 100))

print('Accuracy of prediction (team rating) = {:.2f}%'.format(accuracy_score(compare_results_2['actual_res'], compare_results_2['predict_result']) * 100))

Accuracy of prediction (old match) = 58.60%
Accuracy of prediction (team rating) = 57.67%


In [40]:
# targets # ผลการแข่งขันจริงๆ
print('Total accuracy of prediction = {:.2f}%'.format(accuracy_score(compare_results_3['actual_res'], compare_results_3['predict_result']) * 100))

Total accuracy of prediction = 59.07%


## Predict next week from 2 model

#### From first model

In [41]:
compare_results_1_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_1_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_1_1 = pd.concat([compare_results_1_1, predicted_table_1], axis=1)
compare_results_1_1 = compare_results_1_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_1_1 = compare_results_1_1.rename(columns={'predict_res': 'predict_result'})
compare_results_1_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7298,30,NaN,NaN,West Bromwich Albion,win,Leicester City,0.239910,0.350904,0.409186,draw
1,7290,30,NaN,NaN,Bournemouth,lose,Tottenham Hotspur,0.235408,0.584776,0.179815,draw
2,7291,30,NaN,NaN,Arsenal,win,Watford,0.164669,0.119131,0.716200,draw
3,7292,30,NaN,NaN,Chelsea,win,Crystal Palace,0.220522,0.102860,0.676618,draw
4,7293,30,NaN,NaN,Everton,win,Brighton,0.347183,0.235555,0.417262,draw
5,7294,30,NaN,NaN,Huddersfield,win,Swansea City,0.256277,0.313076,0.430647,draw
6,7295,30,NaN,NaN,Manchester United,win,Liverpool,0.192134,0.297309,0.510557,draw
7,7296,30,NaN,NaN,Newcastle United,win,Southampton,0.201030,0.370543,0.428427,draw
8,7297,30,NaN,NaN,Stoke City,lose,Manchester City,0.252134,0.460408,0.287458,draw
9,7299,30,NaN,NaN,West Ham United,win,Burnley,0.186607,0.120524,0.692869,draw


#### From second model

In [42]:
compare_results_2_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2_1 = pd.concat([compare_results_2_1, predicted_table_2_1], axis=1)
compare_results_2_1 = compare_results_2_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2_1 = compare_results_2_1.rename(columns={'predict_res': 'predict_result'})
compare_results_2_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7298,30,NaN,NaN,West Bromwich Albion,win,Leicester City,0.263656,0.305880,0.430464,draw
1,7290,30,NaN,NaN,Bournemouth,lose,Tottenham Hotspur,0.271197,0.449860,0.278943,draw
2,7291,30,NaN,NaN,Arsenal,win,Watford,0.234988,0.202629,0.562382,draw
3,7292,30,NaN,NaN,Chelsea,win,Crystal Palace,0.224753,0.176578,0.598669,draw
4,7293,30,NaN,NaN,Everton,win,Brighton,0.237416,0.210267,0.552317,draw
5,7294,30,NaN,NaN,Huddersfield,win,Swansea City,0.265191,0.320588,0.414222,draw
6,7295,30,NaN,NaN,Manchester United,win,Liverpool,0.250300,0.246715,0.502985,draw
7,7296,30,NaN,NaN,Newcastle United,win,Southampton,0.267231,0.329505,0.403263,draw
8,7297,30,NaN,NaN,Stoke City,lose,Manchester City,0.271775,0.444466,0.283758,draw
9,7299,30,NaN,NaN,West Ham United,win,Burnley,0.248370,0.242683,0.508947,draw


## Last result

In [43]:
compare_results_3_1 = compare_results_1_1.copy()

# weight data
compare_results_3_1['draw'] = compare_results_1_1['draw'] * model_weight + compare_results_2_1['draw'] * (1 - model_weight)
compare_results_3_1['lose'] = compare_results_1_1['lose'] * model_weight + compare_results_2_1['lose'] * (1 - model_weight)
compare_results_3_1['win'] = compare_results_1_1['win'] * model_weight + compare_results_2_1['win'] * (1 - model_weight)
compare_results_3_1['predict_result'] = compare_results_3_1.apply(predict_home_result, axis=1)
compare_results_3_1.drop('actual_res', axis=1, inplace=True)
compare_results_3_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win
0,7298,30,NaN,NaN,West Bromwich Albion,win,Leicester City,0.244659,0.341899,0.413442
1,7290,30,NaN,NaN,Bournemouth,lose,Tottenham Hotspur,0.242566,0.557793,0.199641
2,7291,30,NaN,NaN,Arsenal,win,Watford,0.178733,0.135831,0.685436
3,7292,30,NaN,NaN,Chelsea,win,Crystal Palace,0.221368,0.117604,0.661028
4,7293,30,NaN,NaN,Everton,win,Brighton,0.325229,0.230497,0.444273
5,7294,30,NaN,NaN,Huddersfield,win,Swansea City,0.258060,0.314579,0.427362
6,7295,30,NaN,NaN,Manchester United,win,Liverpool,0.203767,0.287190,0.509043
7,7296,30,NaN,NaN,Newcastle United,win,Southampton,0.214270,0.362336,0.423394
8,7297,30,NaN,NaN,Stoke City,lose,Manchester City,0.256063,0.457219,0.286718
9,7299,30,NaN,NaN,West Ham United,win,Burnley,0.198959,0.144956,0.656085


### Classification report

In [44]:
""" Old match model """
print(classification_report(compare_results['actual_res'], compare_results['predict_result'], labels=['draw', 'lose', 'win']))

             precision    recall  f1-score   support

       draw       0.37      0.09      0.15       108
       lose       0.60      0.64      0.62       116
        win       0.60      0.82      0.69       206

avg / total       0.54      0.59      0.54       430



In [45]:
""" Team rating model """
print(classification_report(compare_results_2['actual_res'], compare_results_2['predict_result'], labels=['draw', 'lose', 'win']))

             precision    recall  f1-score   support

       draw       0.00      0.00      0.00       108
       lose       0.63      0.51      0.56       116
        win       0.56      0.92      0.70       206

avg / total       0.44      0.58      0.49       430



/Users/opalchp/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
""" Combine model """
print(classification_report(compare_results_3['actual_res'], compare_results_3['predict_result'], labels=['draw', 'lose', 'win']))

             precision    recall  f1-score   support

       draw       0.47      0.06      0.11       108
       lose       0.59      0.63      0.61       116
        win       0.60      0.84      0.70       206

avg / total       0.56      0.59      0.53       430



### Confusion matrix

In [47]:
""" Old match model """
print(confusion_matrix(compare_results['actual_res'], compare_results['predict_result'], labels=['draw', 'lose', 'win']))

[[ 10  25  73]
 [  3  74  39]
 [ 14  24 168]]


In [48]:
""" Team rating model """
print(confusion_matrix(compare_results_2['actual_res'], compare_results_2['predict_result'], labels=['draw', 'lose', 'win']))

[[  0  18  90]
 [  0  59  57]
 [  0  17 189]]


In [49]:
""" Combine model """
print(confusion_matrix(compare_results_3['actual_res'], compare_results_3['predict_result'], labels=['draw', 'lose', 'win']))

[[  7  25  76]
 [  1  73  42]
 [  7  25 174]]
